<center><font size=6>栅格数据处理</font></center>

In [2]:
import os
import glob
import time

from osgeo import gdal

## 打开/关闭栅格数据

In [2]:
# 打开数据
ds = gdal.Open('data/input/LT52000/LT51180382000158BJC02/LT51180382000158BJC02_B1.TIF')

# 关闭数据（非必须）
ds = None

D:\ProgramData\Anaconda3\envs_1\py310\lib\site-packages\osgeo\gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


## 获取元数据

In [3]:
gtif = gdal.Open('data/input/LT52000/LT51180382000158BJC02/LT51180382000158BJC02_B1.TIF')
print(gtif.GetMetadata())

{'AREA_OR_POINT': 'Point'}


In [4]:
def Raster_info(file):
    data = gdal.Open(file)
    driver = gdal.GetDriverByName('GTiff')
    print('Raster Driver : {d}\n'.format(d=driver.ShortName))
    print('影像的波段数: ', data.RasterCount)
    img_width, img_height = data.RasterXSize, data.RasterYSize
    print('影像的列，行数: {r}rows * {c}colums'.format(r=img_width, c=img_height))
    print('栅格数据的空间参考：{}'.format(data.GetGeoTransform()))
    print('投影信息：{}\n'.format(data.GetProjection()))

In [5]:
Raster_info('data/input/LT52000/LT51180382000158BJC02/LT51180382000158BJC02_B1.TIF')

Raster Driver : GTiff

影像的波段数:  1
影像的列，行数: 7891rows * 7201colums
栅格数据的空间参考：(277185.0, 30.0, 0.0, 3621315.0, 0.0, -30.0)
投影信息：PROJCS["WGS 84 / UTM zone 51N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",123],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32651"]]



## 波段融合

定义波段融合函数，方便处理多幅影像

In [7]:
def Band_fusion(in_path, file_name, out_path):
    # 创建输出文件夹
    if not os.path.exists(out_path):
        os.makedirs(out_path)
        
    # 打开数据
    ds1 = gdal.Open(f'{in_path}/{file_name}/{file_name}_B1.TIF')
    ds2 = gdal.Open(f'{in_path}/{file_name}/{file_name}_B2.TIF')
    ds3 = gdal.Open(f'{in_path}/{file_name}/{file_name}_B3.TIF')
    ds4 = gdal.Open(f'{in_path}/{file_name}/{file_name}_B4.TIF')
    ds5 = gdal.Open(f'{in_path}/{file_name}/{file_name}_B4.TIF')
    ds6 = gdal.Open(f'{in_path}/{file_name}/{file_name}_B4.TIF')
    ds7 = gdal.Open(f'{in_path}/{file_name}/{file_name}_B4.TIF')

    # 获取波段
    b1 = ds1.GetRasterBand(1)
    b2 = ds2.GetRasterBand(1)
    b3 = ds3.GetRasterBand(1)
    b4 = ds4.GetRasterBand(1)
    b5 = ds5.GetRasterBand(1)
    b6 = ds6.GetRasterBand(1)
    b7 = ds7.GetRasterBand(1)

    # 创建数据
    driver = gdal.GetDriverByName("GTiff")
    out_ds = driver.Create(f'{out_path}/{file_name}_merge.tif', ds1.RasterXSize, ds1.RasterYSize, 7, gdal.GDT_Byte)

    # 设置坐标和投影
    out_ds.SetGeoTransform(ds1.GetGeoTransform())
    out_ds.SetProjection(ds1.GetProjection())

    ob1 = out_ds.GetRasterBand(1)
    ob2 = out_ds.GetRasterBand(2)
    ob3 = out_ds.GetRasterBand(3)
    ob4 = out_ds.GetRasterBand(4)
    ob5 = out_ds.GetRasterBand(5)
    ob6 = out_ds.GetRasterBand(6)
    ob7 = out_ds.GetRasterBand(7)

    # 写入数据
    ob1.WriteArray(b1.ReadAsArray())
    ob2.WriteArray(b2.ReadAsArray())
    ob3.WriteArray(b3.ReadAsArray())
    ob4.WriteArray(b4.ReadAsArray())
    ob5.WriteArray(b5.ReadAsArray())
    ob6.WriteArray(b6.ReadAsArray())
    ob7.WriteArray(b7.ReadAsArray())

    # 关闭栅格
    out_ds = None 

调用函数融合影像

In [8]:
path = 'data/input/LT52000'  # 原始影像路径
path1 = 'data/output/LT52000'  # 波段融合后影像路径
file_name = os.listdir(path)  # 获取影像名
for file in file_name:
    Band_fusion(path, file, path1)  # 调用函数融合波段

## 影像拼接

In [77]:
files_to_mosaic = glob.glob('data/output/LT52000/*.tif')
files_to_mosaic = [path+"\\"+files[-31:] for files in files_to_mosaic]
files_string = " ".join(files_to_mosaic) # 生成input_files
files_to_mosaic

['E:\\jupyterlab\\data\\output\\LT52000\\LT51180382000158BJC02_merge.tif',
 'E:\\jupyterlab\\data\\output\\LT52000\\LT51180392000158HAJ03_merge.tif',
 'E:\\jupyterlab\\data\\output\\LT52000\\LT51190382000165BJC00_merge.tif',
 'E:\\jupyterlab\\data\\output\\LT52000\\LT51190392000165BJC00_merge.tif']

In [4]:
%run D:\ProgramData\Anaconda3\envs_1\py310\Scripts\gdal_merge.py -o E:\jupyterlab\data\output\Mosaic_LT5.tif -n 0 -of GTiff E:\jupyterlab\data\output\LT52000\LT51180382000158BJC02_merge.tif E:\jupyterlab\data\output\LT52000\LT51180392000158HAJ03_merge.tif E:\jupyterlab\data\output\LT52000\LT51190382000165BJC00_merge.tif

In [2]:
%run D:\ProgramData\Anaconda3\envs_1\py310\Scripts\gdal_merge.py -o E:\jupyterlab\data\output\Mosaic_LT5_1.tif -n 0 -of GTiff E:\jupyterlab\data\output\LT52000\LT51180392000158HAJ03_merge.tif E:\jupyterlab\data\output\LT52000\LT51190392000165BJC00_merge.tif

In [6]:
%run D:\ProgramData\Anaconda3\envs_1\py310\Scripts\gdal_merge.py -o E:\jupyterlab\data\output\Mosaic_LT5_3.tif -n 0 -of GTiff E:\jupyterlab\data\output\LT52000\LT51180382000158BJC02_merge.tif E:\jupyterlab\data\output\LT52000\LT51180392000158HAJ03_merge.tif E:\jupyterlab\data\output\LT52000\LT51190382000165BJC00_merge.tif

In [ ]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

db_connection = {
    "host": "localhost",
    "port": "5432",
    "user": "postgres",
    "password": "123",
    "dbname": "osm"
}
conn = psycopg2.connect(**db_connection) # 连接数据库
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT) #  设置自动提交事务
cursor = conn.cursor() #  创建游标

# 批量修改表名称
try:
    cursor.execute("SELECT relname FROM pg_class WHERE relkind = 'r' AND relname != '^(pg_|sql_|spatial_)';")
    for i in cursor.fetchall():
        cursor.execute(f'ALTER TABLE "{i[0]}" RENAME TO "2022_{i[0]}";')
        print(f'{i[0]}完成')
except:
    print('失败')